#   2. Write a function for calculating VaR using Historical Simulation. 

In [2]:
import yfinance as yf
import pandas as pd
import numpy as np
import scipy.stats as stats
import datetime as dt

C:\Users\koushik\AppData\Roaming\Python\Python311\site-packages\yfinance\base.py:48: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  _empty_series = pd.Series()


## Function to get stock data

In [3]:
# Function to get stock data
def get_stock_data(stockList, startDate, endDate):
    stockData = yf.download(stockList, start=startDate, end=endDate)
    stockData = stockData['Adj Close']
    return stockData

In [4]:
stock_list = ['DLF.NS', 'NTPC.NS', 'HDFCBANK.NS']
wtgs = [0.4, 0.2, 0.4]
df = get_stock_data(stockList=stock_list, startDate='2017-09-15', endDate='2019-10-1')
data = df.copy(deep=True)

[*********************100%%**********************]  3 of 3 completed


## Function to calculate Log Returns, Mean and Standard Deviation from the data

In [5]:
def get_log_returns(data, weights):
    # Calculate Log Returns
    log_returns = np.log(data).diff()
    log_returns = log_returns[stock_list]
    log_returns.dropna(inplace=True)
    
    # Calculate Weighted returns
    portReturns = np.sum(log_returns * weights, axis=1)
    portReturns = portReturns.sort_index(ascending=False)
    portReturns.name='Port'
    portReturns = portReturns.to_frame()

    return portReturns

In [6]:
returns = get_log_returns(data, wtgs)
returns

,Port
Date,
2019-09-30,-0.011662
2019-09-27,-0.007095
2019-09-26,0.015423
2019-09-25,-0.030002
2019-09-24,-0.003731
...,...
2017-09-22,-0.031688
2017-09-21,-0.011671
2017-09-20,-0.003607


## Funtion to calculate Historical VaR of the Data

In [7]:
def historical_VaR(returns, confInterval):
    returns = returns.Port*1e6
    return np.percentile(returns, confInterval)

In [8]:
# 99% Confidence Interval
confInterval = 0.99
historicalVaR = historical_VaR(returns, confInterval)

print("Historical VaR of " + str(confInterval * 100) + " Confidence Interval is : " + str(historicalVaR))

Historical VaR of 99.0 Confidence Interval is : -37109.3410588473


## Historical VaR of 99.0 Confidence Interval is : -37109.22606175549